# Clase 3 – Ingesta y Capa Bronce

En esta notebook se inicia la construcción del pipeline de datos meteorológicos, trabajando con los archivos crudos provistos por el SMN.


## Importar las librerías necesarias

In [7]:
import pandas as pd
import numpy as np
import re
import os
import json
from glob import glob
from pathlib import Path

print("Importación de librerías completada.")

Importación de librerías completada.


## Configuración de paths y carpetas del proyecto

In [8]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'

# Crear carpetas si no existen
for path in [BRONCE_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print("Iniciación de carpetas del proyecto completada.")

Iniciación de carpetas del proyecto completada.


## Lectura del archivo de estaciones

In [9]:
# Ruta del archivo
archivo_estaciones = RAW_DIR / 'estaciones' / 'estaciones_smn.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_estaciones, "r", encoding="latin1") as f:
    lines = f.readlines()[2:]

# Expresión regular para extraer campos:
pattern = re.compile(
    r"^(?P<nombre>.+?)\s{2,}(?P<provincia>.+?)\s{2,}(?P<lat_gr>-?\d+)\s+(?P<lat_min>\d+)\s+(?P<lon_gr>-?\d+)\s+(?P<lon_min>\d+)\s+(?P<altura_m>\d+)\s+(?P<numero>\d+)\s+(?P<numero_oaci>\S+)\s*$"
)

# Extraer los datos
data = []
for line in lines:
    match = pattern.match(line)
    if match:
        data.append(match.groupdict())

# Crear DataFrame
df_estaciones = pd.DataFrame(data)

# Conversión de tipos
df_estaciones[['lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero']] = df_estaciones[[
    'lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero'
]].apply(pd.to_numeric)

# Cargar las provincias
provincias_unicas = df_estaciones['provincia'].str.strip().str.upper().unique()

# Imprimir la cantidad de estaciones registradas
print("Estaciones cargadas:", len(df_estaciones))

# Imprimir la cantidad de provincias registradas
print("Cantidad de provincias:", len(provincias_unicas))

# Imprimir las provincias
print("Provincias disponibles:", provincias_unicas)


Estaciones cargadas: 117
Cantidad de provincias: 26
Provincias disponibles: ['ANTARTIDA' 'BUENOS AIRES' 'CAPITAL FEDERAL' 'CATAMARCA' '' 'CHACO'
 'CHUBUT' 'CORDOBA' 'CORRIENTES' 'ENTRE RIOS' 'FORMOSA' 'JUJUY' 'LA PAMPA'
 'LA RIOJA' 'MENDOZA' 'MISIONES' 'NEUQUEN' 'RIO NEGRO' 'SALTA' 'SAN JUAN'
 'SAN LUIS' 'SANTA CRUZ' 'SANTA FE' 'SANTIAGO DEL ESTERO'
 'TIERRA DEL FUEGO' 'TUCUMAN']


## Selección de estaciones. 

### Para el desarrollo del trabajo se utilizarán las estaciones ubicadas en la provincia de Misiones

In [10]:
# Ingresar el nombre de la provincia con la que se va a trabajar
provincia = 'MISIONES'

df_provincia = df_estaciones[df_estaciones['provincia'].str.upper() == provincia]
df_provincia[['nombre', 'provincia', 'numero', 'numero_oaci']]

,nombre,provincia,numero,numero_oaci
77,BERNARDO DE IRIGOYEN AERO,MISIONES,87163,SATI
78,IGUAZU AERO,MISIONES,87097,SARI
79,OBERA,MISIONES,87187,SATO
80,POSADAS AERO,MISIONES,87178,SARP


## Filtrar las estaciones que correspondan a la provincia seleccionada

In [11]:
# Se selecciona una fecha para visualizar los datos
archivo_dato = RAW_DIR / 'datohorario' / 'datohorario20250531.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_dato, "r", encoding="latin1") as f:
    lines = f.readlines()

# Detectar columnas separadas por múltiples espacios
columnas = re.split(r"\s{2,}", lines[0].strip())

# Leer datos
data = [
    re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
    for line in lines[1:]
    if len(line.strip()) > 0 and not line.isspace()
]

# Crear DataFrame con columnas originales
df_dato = pd.DataFrame(data, columns=columnas)
df_dato.columns = df_dato.columns.str.strip()
df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()

# Filtrar por estaciones
nombres_provincia = df_provincia["nombre"].str.strip().unique()
df_provincia_dia = df_dato[df_dato["NOMBRE"].isin(nombres_provincia)]

# Crear copia y convertir tipos SOLO para impresión de tipos correctos
df_tipos = df_provincia_dia.copy()
df_tipos["FECHA"] = pd.to_datetime(df_tipos["FECHA"], format="%d%m%Y", errors="coerce").dt.date
df_tipos["HORA"] = pd.to_numeric(df_tipos["HORA"], errors="coerce").astype("Int64")
df_tipos["TEMP"] = pd.to_numeric(df_tipos["TEMP"], errors="coerce")
df_tipos["HUM"] = pd.to_numeric(df_tipos["HUM"], errors="coerce")
df_tipos["PNM"] = pd.to_numeric(df_tipos["PNM"], errors="coerce")
df_tipos["DD"] = pd.to_numeric(df_tipos["DD"], errors="coerce").astype("Int64")
df_tipos["FF"] = pd.to_numeric(df_tipos["FF"], errors="coerce").astype("Int64")

# Mostrar todos los resultados
print(df_provincia_dia.to_string(index=False))
print()
print("Columnas:", df_dato.columns.tolist())
print("Tipos de dato:")
print(df_tipos.dtypes)


   FECHA HORA TEMP HUM    PNM  DD FF       NOMBRE
31052025    0  5.6  95 1022.3 270  6  IGUAZU AERO
31052025    1  5.5  95 1021.9 230  9  IGUAZU AERO
31052025    2  5.3 100 1021.8 230  6  IGUAZU AERO
31052025    3  5.3 100 1021.5   0  0  IGUAZU AERO
31052025    4  4.8 100 1021.4   0  0  IGUAZU AERO
31052025    5  4.6 100 1021.3  90  6  IGUAZU AERO
31052025    6  4.4 100 1021.5  90  7  IGUAZU AERO
31052025    7  4.7 100 1022.1  90  9  IGUAZU AERO
31052025    8  5.0 100 1023.2   0  0  IGUAZU AERO
31052025    9  9.2  95 1023.4   0  0  IGUAZU AERO
31052025   10 10.1  89 1024.2   0  0  IGUAZU AERO
31052025   11 13.5  83 1023.8  50  6  IGUAZU AERO
31052025   12 14.8  76 1023.1   0  0  IGUAZU AERO
31052025   13 16.0  73 1022.0   0  0  IGUAZU AERO
31052025   14 17.8  66 1021.2   0  0  IGUAZU AERO
31052025   15 18.0  64 1020.6   0  0  IGUAZU AERO
31052025   16 18.2  61 1020.4   0  0  IGUAZU AERO
31052025   17 17.6  71 1020.6   0  0  IGUAZU AERO
31052025   18 15.2  85 1021.3   0  0  IGUAZU AERO


## Procesamiento por estación y por fecha (con limpieza y reporte resumen)

In [12]:
# Crear carpeta de salida si no existe
BRONCE_DIR.mkdir(parents=True, exist_ok=True)

# Buscar todos los archivos datohorario disponibles
archivos_datos = sorted(glob(str(RAW_DIR / "datohorario" / "datohorario*.txt")))

errores_globales = 0

for archivo in archivos_datos:
    try:
        with open(archivo, encoding="latin1") as f:
            raw_lines = f.readlines()

        header = raw_lines[0].strip()
        columnas = re.split(r"\s{2,}", header)

        data = [
            re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
            for line in raw_lines[1:]
            if len(line.strip()) > 0 and not line.isspace()
        ]

        df_dato = pd.DataFrame(data, columns=columnas)
        df_dato.columns = df_dato.columns.str.strip()
        df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()

        # Filtrar por estaciones según la provincia
        df_provincia = df_dato[df_dato["NOMBRE"].isin(nombres_provincia)]

        # Obtener fecha
        fecha = Path(archivo).stem.replace("datohorario", "")

        # Guardar archivos por estación
        for nombre in nombres_provincia:
            nombre_clean = nombre.lower().replace(" ", "_")
            df_estacion = df_provincia[df_provincia["NOMBRE"] == nombre]

            if not df_estacion.empty:
                path_estacion = BRONCE_DIR / nombre_clean
                path_estacion.mkdir(parents=True, exist_ok=True)

                # Archivos de salida
                archivo_parquet = path_estacion / f"{fecha}.parquet"
                archivo_csv = path_estacion / f"{fecha}.csv"
                archivo_txt = path_estacion / f"{fecha}.txt"
                
                # Guardar en distintos formatos
                df_estacion.to_parquet(archivo_parquet, index=False)
                df_estacion.to_csv(archivo_csv, index=False)

                with open(archivo_txt, "w", encoding="latin1") as f:
                    f.write("  ".join(df_estacion.columns) + "\n")
                    for _, row in df_estacion.iterrows():
                        f.write("  ".join(map(str, row.values)) + "\n")

    except Exception as e:
        errores_globales += 1
        continue

# Reporte final
print("Proceso completado.")
print(f"Días procesados: {len(archivos_datos)}")
print(f"Errores al procesar archivos: {errores_globales}")

Proceso completado.
Días procesados: 391
Errores al procesar archivos: 0
